The end goal of making these decision trees is to model the decision making process for each coach. To help us better understand the individual trees, we will start by making an "average" baseline tree that is built using data from every coach. This classification tree will predict whether or not a coach decides to go for it on a given 4th down scenario. It may help to try this using every 4th down play, as well as with some filters imposed on the data to extract more informative, higher leverage scenarios.

# Python


In [ ]:
import pandas as pd
import seaborn as sns 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

## Basic Data Exploration

We will take another look at our data just so we are familiar with the parameters we are working with.


In [ ]:
df = pd.read_csv("data/df.csv").drop(["coach", "Unnamed: 0"], axis=1)
summary = pd.DataFrame({"dtypes": df.dtypes, "min" : df.min(), "mean" : df.mean(), "max" : df.max()})
print(summary)

In [ ]:
count_0 = sum(df["go"] == 0)
count_1 = sum(df["go"] == 1)
total = count_0 + count_1

print("Number of not go points:", count_0, count_0/total)
print("Number of go points:", count_1, count_1/total)

## Isolate Inputs/Output


In [ ]:
# Split data into features and target
X = df.iloc[:,[0, 1, 2, 3, 11]]
Y = df["go"]

In [ ]:
# Split data into train and test sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=621)

# Check size of train and test splits
print(type(x_train), x_train.shape)
print(type(y_train), y_train.shape)
print(type(x_test), x_test.shape)
print(type(y_test), y_test.shape)

## Training the Model


In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(random_state = 621, max_depth = 5)
model = clf.fit(x_train, y_train)

## Check the Model


In [ ]:
yp_train = model.predict(x_train)
yp_test = model.predict(x_test)

def confusion_plot(y_data, y_pred):
    cm = confusion_matrix(y_data, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                  display_labels=model.classes_)
    print("ACCURACY: ", accuracy_score(y_data, y_pred))
    print("NEGATIVE RECALL (Y=0): ", recall_score(y_data, y_pred, pos_label=0))
    print("NEGATIVE PRECISION (Y=0): ", precision_score(y_data, y_pred, pos_label=0))
    print("POSITIVE RECALL (Y=1): ", recall_score(y_data, y_pred, pos_label=1))
    print("POSITIVE PRECISION (Y=1): ", precision_score(y_data, y_pred, pos_label=1))
    print(cm)
    disp.plot()
    plt.show()
    
print("------TRAINING------")
confusion_plot(y_train,yp_train)
print("------TEST------")
confusion_plot(y_test,yp_test)

Negative recall and precision are very good, meaning our model is good at predicting when coaches will not go for it, which makes sense. Positive recall and precision, on the other hand, are not nearly as good. It is much harder to predict when coaches will go for it than when they will not go for it.

## Visualize the Tree


In [ ]:
def plot_tree(model,X,Y):
    fig = plt.figure(figsize=(25,20))
    _ = tree.plot_tree(model,
                    feature_names=X.columns,
                    class_names=[str(class_) for class_ in Y.unique()], 
                        filled=True)
    plt.show()

plot_tree(model, X, Y)

## Hyper-parameter tuning


In [ ]:
test_results=[]
train_results=[]

for num_layer in range(1,10):
    model = tree.DecisionTreeClassifier(max_depth=num_layer)
    model = model.fit(x_train, y_train)

    yp_train=model.predict(x_train)
    yp_test=model.predict(x_test)

    # print(y_pred.shape)
    test_results.append([num_layer,
    accuracy_score(y_test, yp_test),
    recall_score(y_test, yp_test,pos_label=0),
    recall_score(y_test, yp_test,pos_label=1)])
    
    train_results.append([num_layer,
    accuracy_score(y_train, yp_train),
    recall_score(y_train, yp_train,pos_label=0),
    recall_score(y_train, yp_train,pos_label=1)])

In [ ]:
plt.cla()
ax = sns.lineplot(x=np.asarray(test_results)[:,0],
                y=np.asarray(test_results)[:,1],
                color="red",
                marker="o",
                label="test")
sns.lineplot(x=np.asarray(train_results)[:,0],
                y=np.asarray(train_results)[:,1],
                color="blue",
                marker="o",
                label="train")
ax.set(xlabel="Number of layers in the decision tree (max_depth)", ylabel="ACCURACY")
plt.legend()
plt.show()

plt.cla()
ax = sns.lineplot(x=np.asarray(test_results)[:,0],
                y=np.asarray(test_results)[:,3],
                color="red",
                marker="o",
                label="test")
sns.lineplot(x=np.asarray(train_results)[:,0],
                y=np.asarray(train_results)[:,3],
                color="blue",
                marker="o",
                label="train")
ax.set(xlabel="Number of layers in the decision tree (max_depth)", ylabel="POSITIVE RECALL (Y=1)")
plt.legend()
plt.show()

Looks like 4 layers is the optimal `max_depth` parameter.
